# Save NWM to Parquet using Bags in parallel 

In [1]:
import json
from os.path import basename, join
import os 
import shutil
from os import makedirs

import dask.bag as db
import dask.dataframe as dd
from dask.distributed import Client
import xarray as xr
import fsspec
from rechunker import rechunk
import s3fs
import fsspec

%matplotlib inline

def get_json(uri):
    with fsspec.open(uri) as fd:
        return json.load(fd)

In [2]:
# Connect to existing cluster using cluster.name

# This constant needs to be set!
cluster_name = ''
gateway = Gateway()
cluster = gateway.connect(cluster_name)
client = cluster.get_client()

In [7]:
# one year subset of data
zarr_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/zarr/07-06-2022b/nwm-subset.zarr/'
out_root_uri = 's3://azavea-noaa-hydro-data/esip-experiments/datasets/reanalysis-chrtout/parquet/lf/07-11-2022a'

zarr_ds = xr.open_zarr(zarr_uri)
zarr_ds

<xarray.Dataset>
Dimensions:     (feature_id: 122256, time: 8784)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
  * feature_id  (feature_id) int32 1748535 1748537 ... 932080037 932080038
    gage_id     (feature_id) |S15 dask.array<chunksize=(1000,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(1000,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(1000,), meta=np.ndarray>
  * time        (time) datetime64[ns] 1990-01-01 ... 1991-01-01T23:00:00
Data variables:
    crs         |S1 ...
    streamflow  (time, feature_id) float64 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(1000, 1000), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [8]:
def save_parquet_segment(bounds):
    start_ind, end_ind = bounds
    sub_ds = zarr_ds.sel(feature_id=zarr_ds.feature_id[start_ind:end_ind])
    # Using `to_dask_dataframe` works, but seems to get bottlenecked running almost everything on a single worker.
    # Using `to_dataframe` runs much faster, but fails when mapping > 32 items in the cells below.
    df = sub_ds.to_dask_dataframe()
    df.to_parquet(join(out_root_uri, f'nwm-{start_ind}-{end_ind}.parquet'), engine='pyarrow')
    # TODO: delete stuff explicitly

In [ ]:
features_per_store = 2000
feature_bounds = [
    (start_ind, min(start_ind + features_per_store, zarr_ds.feature_id.shape[0]))
    for start_ind in range(0, zarr_ds.feature_id.shape[0], features_per_store)]

In [10]:
b = db.from_sequence(feature_bounds, npartitions=len(feature_bounds))
b = b.map(save_parquet_segment)
results_bag = b.compute()

In [22]:
ddf = dd.read_parquet(out_root_uri)

In [23]:
ddf

,feature_id,time,elevation,gage_id,latitude,longitude,order,crs,streamflow,velocity,dir0
npartitions=1107,,,,,,,,,,,
,int32,datetime64[ns],float32,object,float32,float32,int32,object,float64,float64,category[known]
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
